In [1]:
import torch
import pypose as pp

# 1. Intialization

In [2]:
a = pp.so3(torch.randn(2,3))
x = pp.identity_SE3(2,1)
y = pp.randn_se3(2,2)
print('a:', a, '\nx.shape:', x.shape, '\nx.gshape:', x.gshape)
print(x.gview(2))
print(y)

a: so3Type Group:
tensor([[-0.4009,  1.0486, -0.0641],
        [ 0.6245, -0.5652,  0.9019]]) 
x.shape: torch.Size([2, 1, 7]) 
x.gshape: torch.Size([2, 1])
SE3Type Group:
tensor([[0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 1.]])
se3Type Group:
tensor([[[-0.5171, -1.1020, -0.6251, -0.5961,  2.9070,  0.2051],
         [-0.4160, -0.8013,  0.3193, -0.7172,  1.4841, -0.1545]],

        [[ 0.9216,  1.2305, -1.0666,  1.1960, -1.3424, -0.4081],
         [ 0.6565,  0.4741, -0.8166, -1.2900,  0.5438, -1.8964]]])


### All arguments in PyTorch are supported

In [3]:
a = pp.randn_SO3(3, device="cuda:0", dtype=torch.double, requires_grad=True)
b = pp.identity_like(a, device="cpu")
a, b
t = a.float()
a, t

(SO3Type Group:
 tensor([[-0.1872, -0.0170, -0.1539,  0.9700],
         [ 0.0393, -0.5652, -0.1045,  0.8174],
         [ 0.3654,  0.1013,  0.5684,  0.7301]], device='cuda:0',
        dtype=torch.float64, requires_grad=True),
 SO3Type Group:
 tensor([[-0.1872, -0.0170, -0.1539,  0.9700],
         [ 0.0393, -0.5652, -0.1045,  0.8174],
         [ 0.3654,  0.1013,  0.5684,  0.7301]], device='cuda:0',
        grad_fn=<AliasBackward0>))

# 2. Slicing and Shaping

In [4]:
A = pp.randn_SO3(2,2)
B = pp.randn_SO3(2,1)
C = torch.cat([A,B], dim=1)         # Tensor cat
C[0,1] = pp.randn_SO3(1)            # Slicing set
D = C[1,:].Log()                    # Slicing get
E, F = torch.split(C, [1,2], dim=1) # Tensor split
print('A:', A.gshape)
print('B:', B.gshape)
print('C:', C.gshape)
print('D:', D.gshape)
print('E:', E.gshape)
print('F:', F.gshape)

A: torch.Size([2, 2])
B: torch.Size([2, 1])
C: torch.Size([2, 3])
D: torch.Size([3])
E: torch.Size([2, 1])
F: torch.Size([2, 2])


# 3. Basic Operations

In [5]:
(x * y.Exp()).Inv().Log()

se3Type Group:
tensor([[[ 0.5171,  1.1020,  0.6251,  0.5961, -2.9070, -0.2051],
         [ 0.4160,  0.8013, -0.3193,  0.7172, -1.4841,  0.1545]],

        [[-0.9216, -1.2305,  1.0666, -1.1960,  1.3424,  0.4081],
         [-0.6565, -0.4741,  0.8166,  1.2900, -0.5438,  1.8964]]])

# 4. Adjoint Transforms

In [6]:
X = pp.randn_Sim3(6, dtype=torch.double)
a = pp.randn_sim3(6, dtype=torch.double)
b = X.AdjT(a)
print((X * b.Exp() - a.Exp() * X).abs().mean() < 1e-7)

tensor(True)


In [7]:
X = pp.randn_SE3(8)
a = pp.randn_se3(8)
b = X.Adj(a)
print((b.Exp() * X - X * a.Exp()).abs().mean() < 1e-7)

tensor(True)


# 5. Grdients

In [8]:
x = pp.randn_so3(3, sigma=0.1, requires_grad=True, device="cuda")
assert x.is_leaf
loss = (x.Exp().Log()**2).sin().sum() # Just test, No physical meaning
loss.backward()
y = x.detach()
loss, x.grad, x, y

(tensor(0.0827, device='cuda:0', grad_fn=<AliasBackward0>),
 tensor([[ 0.1487, -0.2074,  0.2507],
         [ 0.0400,  0.2287, -0.1283],
         [ 0.2544,  0.2409,  0.0989]], device='cuda:0'),
 so3Type Group:
 tensor([[ 0.0743, -0.1037,  0.1254],
         [ 0.0200,  0.1143, -0.0641],
         [ 0.1272,  0.1205,  0.0494]], device='cuda:0', requires_grad=True),
 so3Type Group:
 tensor([[ 0.0743, -0.1037,  0.1254],
         [ 0.0200,  0.1143, -0.0641],
         [ 0.1272,  0.1205,  0.0494]], device='cuda:0'))

# 6. Test a Module

In [9]:
from torch import nn

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

class TestNet(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.weight = pp.Parameter(pp.randn_so3(n))

    def forward(self, x):
        return self.weight.Exp() * x


n,epoch = 4, 5
net = TestNet(n).cuda()

optimizer = torch.optim.SGD(net.parameters(), lr = 0.2, momentum=0.9)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[2,4], gamma=0.5)

print("Before Optimization:\n", net.weight)
for i in range(epoch):
    optimizer.zero_grad()
    inputs = pp.randn_SO3(n).cuda()
    outputs = net(inputs)
    loss = outputs.abs().sum()
    loss.backward()
    optimizer.step()
    scheduler.step()
    print(loss)

print("Parameter:", count_parameters(net))
print("After Optimization:\n", net.weight)

Before Optimization:
 so3Type Group:
Parameter containing:
tensor([[ 0.6965,  0.9209,  0.8660],
        [ 1.6618,  0.1517, -1.1111],
        [ 0.2451,  0.7669,  1.2152],
        [-0.7460,  0.1278,  0.5047]], device='cuda:0', requires_grad=True)
tensor(7.2796, device='cuda:0', grad_fn=<AliasBackward0>)
tensor(6.9272, device='cuda:0', grad_fn=<AliasBackward0>)
tensor(6.6190, device='cuda:0', grad_fn=<AliasBackward0>)
tensor(6.1722, device='cuda:0', grad_fn=<AliasBackward0>)
tensor(7.0664, device='cuda:0', grad_fn=<AliasBackward0>)
Parameter: 12
After Optimization:
 so3Type Group:
Parameter containing:
tensor([[-0.5233,  0.1666, -0.1501],
        [ 0.7642, -0.4101,  0.4685],
        [-0.0640, -0.0837, -0.2574],
        [-0.3928, -0.7736, -0.3618]], device='cuda:0', requires_grad=True)
